# Optimize Tensorflow Pipeline Performance: prefetch & cache 

In [1]:
#import paskages 
import numpy as np
import time 
import tensorflow as tf

In [29]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [32]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [34]:
%%timeit
benchmark(FileDataset())

331 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
benchmark(FileDataset().prefetch(3))

340 ms ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

339 ms ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
dataset= tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [46]:
dataset= dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [47]:
dataset= dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [49]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [50]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.37 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
